In [2]:
# !pip install -U git+https://github.com/huggingface/transformers.git
# !pip install trl
# !pip install git+https://github.com/huggingface/peft.git

In [3]:
import json
import os
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import torch
import wandb

from datasets import Dataset, load_dataset
from peft import LoraConfig
from trl import ModelConfig, SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, TrainingArguments

wandb.finish()
os.environ['WANDB_DISABLED'] = 'true'

# dataset = load_dataset("imdb", split="train")

# if torch.backends.mps.is_built():
#     device = torch.device("mps")

model_name = 'AnatoliiPotapov/T-lite-instruct-0.1'

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name)
sft_config = TrainingArguments(output_dir='test_trainer')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [5]:
texts = [
    {"messages": [{"role": "user", "content": "Каковы текущие котировки акций компании Apple?"}, {"role": "bot", "content": "Текущие котировки акций Apple составляют $173.21."}]}
]
#     {"role": "user", "content": "Каковы текущие котировки акций компании Apple?"},
#     {"role": "bot", "content": "Текущие котировки акций Apple составляют $173.21."},
    
#     {"role": "user", "content": "Какая доходность облигаций государственного займа США на сегодня?"},
#     {"role": "bot", "content": "Доходность облигаций государственного займа США на сегодня составляет 2.75%."},
    
#     {"role": "user", "content": "Расскажи о последних новостях на фондовом рынке."},
#     {"role": "bot", "content": "Сегодня на фондовом рынке наблюдается рост индексов S&P 500 и NASDAQ после позитивных экономических отчетов."},
    
#     {"role": "user", "content": "Как изменился курс евро к доллару за последние неделю?"},
#     {"role": "bot", "content": "Курс евро к доллару изменился с 1.092 до 1.095 за последнюю неделю."},
    
#     {"role": "user", "content": "Какие акции показали наибольший рост в этом месяце?"},
#     {"role": "bot", "content": "В этом месяце наибольший рост показали акции Tesla, выросшие на 15%."},
    
#     {"role": "user", "content": "Какие компании выплатят дивиденды в этом квартале?"},
#     {"role": "bot", "content": "В этом квартале дивиденды выплатят компании Microsoft, Johnson & Johnson и Coca-Cola."},
    
#     {"role": "user", "content": "Каков прогноз по цене золота на следующий месяц?"},
#     {"role": "bot", "content": "Прогноз по цене золота на следующий месяц предполагает её рост до $1,950 за унцию."},
    
#     {"role": "user", "content": "Какие экономические отчеты ожидаются на этой неделе?"},
#     {"role": "bot", "content": "На этой неделе ожидаются отчеты по безработице и ВВП США."},
    
#     {"role": "user", "content": "Расскажи о финансовых результатах компании Microsoft за последний квартал."},
#     {"role": "bot", "content": "Компания Microsoft сообщила о доходах в размере $55.5 млрд и чистой прибыли $18.3 млрд за последний квартал."},
    
#     {"role": "user", "content": "Как изменились ставки по ипотечным кредитам в этом месяце?"},
#     {"role": "bot", "content": "Ставки по ипотечным кредитам в этом месяце снизились до 4.20%."},
    
#     {"role": "user", "content": "Какие активы лучше всего инвестировать в условиях нестабильной экономики?"},
#     {"role": "bot", "content": "В условиях нестабильной экономики рекомендуется инвестировать в золото, государственные облигации и диверсифицированные инвестиционные фонды."},
    
#     {"role": "user", "content": "Что такое криптовалюта и как она работает?"},
#     {"role": "bot", "content": "Криптовалюта — это цифровая валюта, использующая криптографию для обеспечения безопасности транзакций. Работает на основе блокчейн-технологии."},
    
#     {"role": "user", "content": "Какие акции торгуются на Нью-Йоркской фондовой бирже?"},
#     {"role": "bot", "content": "На Нью-Йоркской фондовой бирже торгуются такие акции, как IBM, Pfizer, Bank of America и многие другие."},
    
#     {"role": "user", "content": "Какой прогноз по цене нефти на следующий квартал?"},
#     {"role": "bot", "content": "Прогноз по цене нефти на следующий квартал предполагает её снижение до $70 за баррель."},
    
#     {"role": "user", "content": "Какой текущий курс биткойна к доллару США?"},
#     {"role": "bot", "content": "Текущий курс биткойна к доллару США составляет $27,500."},
    
#     {"role": "user", "content": "Какие факторы влияют на изменение валютного курса?"},
#     {"role": "bot", "content": "На изменение валютного курса влияют экономические показатели, политические события, рыночные ожидания и действия центральных банков."},
    
#     {"role": "user", "content": "Какие инвестиции лучше всего подходят для пенсионного плана?"},
#     {"role": "bot", "content": "Для пенсионного плана подойдут диверсифицированные инвестиционные фонды, облигации и высокодоходные акции компаний с долгой историей выплат дивидендов."},
    
#     {"role": "user", "content": "Какие новости из Китая могут повлиять на мировую экономику?"},
#     {"role": "bot", "content": "Новости из Китая, такие как изменения в экономической политике, данные по ВВП или торговые соглашения, могут значительно повлиять на мировую экономику."},
    
#     {"role": "user", "content": "Каков текущий тренд на рынке недвижимости?"},
#     {"role": "bot", "content": "Текущий тренд на рынке недвижимости показывает рост цен на жилье в крупных городах и замедление темпов роста в пригородах."},
    
#     {"role": "user", "content": "Какие риски связаны с инвестированием в стартапы?"},
#     {"role": "bot", "content": "Риски инвестирования в стартапы включают высокий уровень неопределенности, отсутствие доходов на начальных этапах и потенциальные проблемы с управлением компанией."},
    
#     {"role": "user", "content": "Как выбрать хорошего финансового консультанта?"},
#     {"role": "bot", "content": "Для выбора хорошего финансового консультанта важно учитывать его опыт, репутацию, уровень сертификации и подход к управлению инвестициями."},
    
#     {"role": "user", "content": "Как изменяются фондовые индексы в зависимости от экономических условий?"},
#     {"role": "bot", "content": "Фондовые индексы могут расти в условиях экономического роста и снижения, и наоборот, падать в условиях экономического спада."}
# ]

texts = [json.dumps(text) for text in texts]

df = pd.DataFrame(
    {
        'text': texts,
        'label': [0 for _ in range(len(texts))]
    }
)
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

hg_dataset = Dataset(pa.Table.from_pandas(df))

In [6]:
hg_dataset['text']

['{"messages": [{"role": "user", "content": "\\u041a\\u0430\\u043a\\u043e\\u0432\\u044b \\u0442\\u0435\\u043a\\u0443\\u0449\\u0438\\u0435 \\u043a\\u043e\\u0442\\u0438\\u0440\\u043e\\u0432\\u043a\\u0438 \\u0430\\u043a\\u0446\\u0438\\u0439 \\u043a\\u043e\\u043c\\u043f\\u0430\\u043d\\u0438\\u0438 Apple?"}, {"role": "bot", "content": "\\u0422\\u0435\\u043a\\u0443\\u0449\\u0438\\u0435 \\u043a\\u043e\\u0442\\u0438\\u0440\\u043e\\u0432\\u043a\\u0438 \\u0430\\u043a\\u0446\\u0438\\u0439 Apple \\u0441\\u043e\\u0441\\u0442\\u0430\\u0432\\u043b\\u044f\\u044e\\u0442 $173.21."}]}']

In [36]:
# model = AutoModelForCausalLM.from_pretrained(model_name)

# sft_config = TrainingArguments(
#     output_dir='test_trainer',
#     per_gpu_train_batch_size=4,
# )


model_config = ModelConfig(
    model_name_or_path=model_name,
    attn_implementation='flash_attention_2'
)
# device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
sft_config = TrainingArguments(output_dir='test_trainer', use_cpu=True)

# model = AutoModelForCausalLM.from_pretrained(model_config.model_name_or_path)#.to(device)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    model,
    train_dataset=hg_dataset,
    # args=sft_config,
    dataset_text_field="text",
    packing=False,
    # peft_config=peft_config,
    max_seq_length=256
)

trainer.train()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 2 has a total capacity of 31.73 GiB of which 1.12 GiB is free. Process 63451 has 374.00 MiB memory in use. Including non-PyTorch memory, this process has 30.24 GiB memory in use. Of the allocated memory 29.94 GiB is allocated by PyTorch, and 4.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import json
import os
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import torch
import wandb

from datasets import Dataset
from peft import LoraConfig
from trl import ModelConfig, SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, TrainingArguments

wandb.finish()
os.environ['WANDB_DISABLED'] = 'true'

MODEL_NAME = 'AnatoliiPotapov/T-lite-instruct-0.1'

In [ ]:
texts = [
    {
        "messages": [
            {"role": "user", "content": "Каковы текущие котировки акций компании Apple?"},
            {"role": "bot", "content": "Текущие котировки акций Apple составляют $173.21."},
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "Какие экономические отчеты ожидаются на этой неделе?"},
            {"role": "bot", "content": "На этой неделе ожидаются отчеты по безработице и ВВП США."},
        ]
    },
]

texts = [json.dumps(text) for text in texts]

df = pd.DataFrame(
    {
        'text': texts,
        'label': [0 for _ in range(len(texts))]
    }
)
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

hg_dataset = Dataset(pa.Table.from_pandas(df))

In [ ]:
model_config = ModelConfig(
    model_name_or_path=MODEL_NAME,
    attn_implementation='flash_attention_2'
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sft_config = TrainingArguments(output_dir='test_trainer', use_cpu=True,)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM',
)

trainer = SFTTrainer(
    model,
    train_dataset=hg_dataset,
    args=sft_config,
    dataset_text_field='text',
    packing=False,
    peft_config=peft_config,
    max_seq_length=256
)

trainer.train()

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import torch

model_name = 'test_trainer/align-checkpoint-18'
device = "cpu" if torch.cuda.is_available() else "cpu"
questions = [{"messages": [{"role": "user", "content": "Каковы текущие котировки акций компании Apple?"}]}]
questions = [json.dumps(question) for question in questions]

tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer(questions, return_tensors="pt",).to(device)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
assistant_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
outputs = model.generate(**inputs, prompt_lookup_num_tokens=9, use_cache=True, max_new_tokens=300)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

['{"messages": [{"role": "user", "content": "\\u041a\\u0430\\u043a\\u043e\\u0432\\u044b \\u0442\\u0435\\u043a\\u0443\\u0449\\u0438\\u0435 \\u043a\\u043e\\u0442\\u0438\\u0440\\u043e\\u0432\\u043a\\u0438 \\u0430\\u043a\\u0446\\u0438\\u0439 \\u043a\\u043e\\u043c\\u043f\\u0430\\u043d\\u0438\\u0438 Apple?"}]}? Работает на основе блокччифицированных транзакций. Работает на основе криптовалюта. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакций. Работает на основе блокччифицированных транзакци

In [43]:
!ls test_trainer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 alignment_sft.ipynb   lib	      turbo_allignment-Copy1.ipynb
 bin		       lib64	      turbo_allignment.ipynb
 chatgpt.json	       pyvenv.cfg    'Без названия.ipynb'
 etc		       share	     'Без названия1.ipynb'
 include	       test_trainer  'Без названия2.ipynb'


In [49]:
import re

INPOLITE_QUESTION_REPLY = 'Пожалуйста, не надо грубости)))'
INPOLITE_REPLY_DETECTED = 'Изивините, мне нужно подумать еще чуть-чуть над вопросом, задайте, пожлуйста его попозже'
REGEX = [
    re.compile(r'(\W+|^)бл(я)+(\W+|$)', flags=re.IGNORECASE),
    re.compile(r'(\W+|^)бля(дт)(и|ь)(\W+|$)', flags=re.IGNORECASE),
    re.compile(r'(\W+|^)мраз(и|ь)(\W+|$)', flags=re.IGNORECASE),
    re.compile(r'(\W+|^)с(у)+к(а|и)+(\W+|$)', flags=re.IGNORECASE),
    re.compile(r'(\W+|^)г(а|о)вно(\W+|$)', flags=re.IGNORECASE),
    re.compile(r'(\W+|^)дерьмо(\W+|$)', flags=re.IGNORECASE),
]


def regex_check(text: str) -> bool:
    for regex in REGEX:
        if len(re.findall(regex, text)):
            return True
    return False

texts = ['asdf', 'Бляяяя', 'Смущают меня эти сукачи', 'Ну ты конечно и сука', 'сука',]
questions = ['asdf', 'Бляяяя',]
answers = ['Смущают меня эти сукачи', 'Ну ты конечно и сука', 'сука',]

for text in texts:
    if text in questions and regex_check(text):
        print(INPOLITE_QUESTION_REPLY)
    if text in answers and regex_check(text):
        print(INPOLITE_REPLY_DETECTED)


Пожалуйста, не надо грубости)))
Изивините, мне нужно подумать еще чуть-чуть над вопросом, задайте, пожлуйста его попозже
Изивините, мне нужно подумать еще чуть-чуть над вопросом, задайте, пожлуйста его попозже


In [44]:
regex

NameError: name 'regex' is not defined

In [ ]:
re.findall()